# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert 🔠 words to 🔢 vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>

_Embeddings_ are representations of words using vectors. These embeddings can be learned within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representations into a Recurrent Neural Network. 

▶️ Run this cell and make sure the version of 📚 [Gensim - Word2Vec](https://radimrehurek.com/gensim/auto_examples/index.html) you are using is ≥ 4.0!

In [1]:
!pip freeze | grep gensim


gensim==4.2.0


In [2]:
!pip freeze | grep tensorflow


tensorflow==2.14.0
tensorflow-datasets==4.6.0
tensorflow-estimator==2.14.0
tensorflow-io-gcs-filesystem==0.27.0
tensorflow-metadata==1.10.0


# The data

Keras provides many datasets, among which is the IMDB dataset 🎬:
- It is comprised of sentences that are ***movie reviews***. 
- Each of these reviews is related to a score given by the reviewer.

❓ **Question** ❓ First of all, let's load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer can handle it. Otherwise, rerun with a lower number.  

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [41]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)

    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)

        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]

        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]

    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]

    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)


<b><u>Embeddings in the previous challenge</u></b>:

In the previous exercise, we jointly learned a representation for the words, and fed this representation to a RNN, as shown down below 👇: 

<img src="layers_embedding.png" width="400px" />

However, this increases the number of parameters to learn, which slows down and increases the difficulty of convergence!

<b><u>Embeddings in the current challenge</u></b>:

For this reason, we will separate the steps of learning the word representation and feeding it into a RNN. As shown here: 

<img src="word2vec_representation.png" width="400px" />

We will learn the embedding with Word2Vec.

The drawback is indeed that the learned embeddings are not _specifically_ designed for our task. However, learning them independently of the task at hand (sentiment analysis) has some advantages: 
- it is very fast to do in general (with Word2Vec)
- the representation learned by Word2Vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with Word2Vec and see how meaningful it is!

# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 🐶 _dog_ $\rightarrow$ [0.1, -0.3, 0.8]
- 🐱 _cat_ $\rightarrow$ [-1.1, 2.3, 0.7]
- 🍏 _apple_ $\rightarrow$ [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

***What is a "good" numerical representation of words?***

- ***Words with close meanings should be geometrically close in your embedding space!***

    - Look at the following example which represents a bi-dimensional embedding space.

![Embedding](word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! 

[📚 **Gensim**](https://radimrehurek.com/gensim/)  is a great Python package that makes the use of the Word2Vec algorithm easy to implement, fast and accurate (which is not an easy task!).

1. The following code imports Word2Vec from Gensim. 

2. The second line learns the embedding representation of the words thanks to the sentences in `X_train`. 
3. The third line stores the words and their trained embeddings in `wv`.

In [4]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)
wv = word2vec.wv


Let's look at the embedded representation of some words.

You can use `wv` as a dictionary.
For instance, `wv['dog']` will return a representation of `dog` in the embedding space.

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation was not learned). 

In [5]:
# YOUR CODE HERE
# print wv['dog']
print(wv['dog'])


[-0.1807609   0.12769982 -0.17605834  0.2061899  -0.05267848 -0.26463073
 -0.01981148  0.5002563  -0.15181302 -0.19214714 -0.01314572 -0.3154129
 -0.09925079  0.13974877 -0.04145002 -0.17530979  0.13688336 -0.2720389
  0.00509825 -0.230468    0.1581642   0.09465405  0.27702    -0.09574915
 -0.00237179  0.03472857 -0.23945391 -0.07685608 -0.1437299   0.0292024
  0.18173957 -0.03956065  0.14912914 -0.3922356  -0.1501962   0.20316167
  0.17636544 -0.17654577 -0.23681407 -0.29936427 -0.13571756 -0.29030395
 -0.28932533  0.17151712  0.31663138 -0.03468932 -0.147574   -0.09494804
  0.25056407  0.19523892  0.06924687 -0.22742666 -0.21184343 -0.02924782
 -0.25812286  0.22659038  0.18011476 -0.08477756 -0.18579109  0.03626899
  0.1141214  -0.00855926  0.00271248  0.04082457 -0.22036088  0.19351278
 -0.04424405  0.19806731 -0.32195833  0.28096482 -0.21860376  0.25815845
  0.2991817  -0.22555205  0.3462212   0.18995056 -0.03058701  0.11056656
 -0.19212234 -0.11652802 -0.2415831  -0.21730924 -0.24

In [7]:
#wv['cupboard']


❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [9]:
# YOUR CODE HERE
# What is the size of each word vector?

print(wv.vector_size)

# What is the size of the embedding matrix?
print(wv.vectors.shape)


100
(8006, 100)


🧐 How do we know whether this embedding make any sense or not? 

💡 To investigate this question, we will check that words with a close meaning have close representations. 

👉 Let's use the [**`Word2Vec.wv.most_similar`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar) method that, given an input word, displays the "closest" words in the embedding space. If the embedding is well done, then words with similar meanings will have similar representation in the embedding space.

❓ **Question** ❓ Try out the `most_similar` method on different words. 

🧑🏿‍🏫 The quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

In [11]:
# YOUR CODE HERE
# Use 'most_similar' to find the word most similar to 'dog'
wv.most_similar('dog')


[('rush', 0.988609254360199),
 ('bill', 0.9883005023002625),
 ('passion', 0.9875573515892029),
 ("stewart's", 0.9875152707099915),
 ('clean', 0.9872452020645142),
 ('lower', 0.9871172308921814),
 ('river', 0.9866034388542175),
 ('costume', 0.9864587187767029),
 ('ill', 0.9861650466918945),
 ('prize', 0.9860702753067017)]

📚 Similarly to `most_similar` used on words directly, we can use [**`similar_by_vector`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.similar_by_vector) on vectors to do the same thing:

In [12]:
# YOUR CODE HERE
# Use 'smiliar_by_vector' to find the word most similar to the vector of 'dog'
wv.similar_by_vector(wv['dog'])


[('dog', 0.9999999403953552),
 ('rush', 0.988609254360199),
 ('bill', 0.9883005023002625),
 ('passion', 0.9875574111938477),
 ("stewart's", 0.9875153303146362),
 ('clean', 0.9872452020645142),
 ('lower', 0.987117350101471),
 ('river', 0.9866034984588623),
 ('costume', 0.9864587187767029),
 ('ill', 0.9861650466918945)]

# Arithmetic on words

Now, let's perform some mathematical operations on words, i.e. on their vector representations!

As any learned word is represented as a vector, you can do basic arithmetic operations, such as:

$$W2V(good) - W2V(bad)$$

❓ **Question** ❓ Do this mathematical operation and print the result

In [15]:
# YOUR CODE HERE
wv['good'] - wv['bad']


array([ 0.05886775, -0.07186022,  0.37115338,  0.2835915 , -0.04340583,
       -0.24898052, -0.51747835, -0.25909904, -0.05218339, -0.41355294,
        0.35448787,  0.03166044,  0.04046917, -0.05695003, -0.16919258,
        0.4633193 , -0.4130486 ,  0.20511317, -0.23094416,  0.20303047,
       -0.02060062,  0.2248916 ,  0.15179539, -0.02956577,  0.04279265,
       -0.07933415, -0.17793247,  0.44407427, -0.17318669, -0.68467826,
       -0.43308544,  0.13790482,  0.52642554,  0.38015246, -0.07976386,
       -0.26527357,  0.5955947 , -0.3521803 , -0.2522175 ,  0.86343646,
       -0.20212317, -0.0539411 , -0.53172755, -0.20288873,  0.3832693 ,
       -0.12520862,  0.5273888 , -0.12270051, -0.15378812,  0.4166784 ,
       -0.16189575, -0.05676588, -0.79324466, -0.04605806,  0.04394685,
        0.36722082,  0.0979647 , -0.06183581,  0.21457976,  0.37524807,
        0.16584033,  0.22955549, -0.09073424, -0.5127964 ,  0.09894633,
        0.10052121, -0.38674366, -0.19894192,  0.4622585 ,  0.32

Now, imagine for a second that the following equality holds true:

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to:

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be bold of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [16]:
# YOUR CODE HERE
res = wv['good'] - wv['bad'] + wv['stupid']

print(res)


[ 0.06953676 -0.09467506  0.43623865  0.00987467 -0.01712451 -0.73825186
 -0.70005804  0.26255175 -0.5294615  -0.66792965  0.09779644 -0.3705289
  0.2920062   0.26189497  0.00480662  0.09785622  0.02266145 -0.36091697
 -0.3961057  -0.6740526   0.47316214  0.15383987  0.9383228  -0.18224669
 -0.01497594 -0.24352176 -0.467263    0.30711251 -0.34705573 -0.5927397
  0.2559089   0.19441868  0.7251208   0.00692233 -0.25705904  0.2723258
  0.51978415 -0.16299504 -0.3977876   0.17542905 -0.16315521 -0.56362987
 -0.5030546   0.23976141  0.5472778  -0.12910497  0.27677852 -0.44655442
 -0.25359964  0.6052507  -0.00683634 -0.16102584 -0.8627241   0.2387085
  0.03431186  0.58467263  0.17540804  0.007225   -0.4136986  -0.05649191
  0.0762137  -0.07865506  0.1154888  -0.13917065 -0.3545383   0.56989974
 -0.10137194 -0.0415446   0.21670914  0.42947024 -0.60981095  0.07538119
  0.34024698  0.45974797  0.43128902  0.02855864 -0.02130335 -0.4279337
 -0.1777214   0.14248966  0.3239048  -0.5249394  -0.7676

We said earlier, that for any vector it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vectors of `res`

💡 _Hint_: `similar_by_vector`

In [17]:
# YOUR CODE HERE
# Closest vectors of 'res'
wv.similar_by_vector(res)


[('nice', 0.7842668890953064),
 ('great', 0.7573084235191345),
 ('decent', 0.7475720047950745),
 ('baldwin', 0.7323789000511169),
 ('good', 0.7286096811294556),
 ('gunslinger', 0.7187718749046326),
 ('alexandra', 0.7180507779121399),
 ('potential', 0.7149003744125366),
 ('wonderful', 0.712282121181488),
 ('written', 0.7061649560928345)]

Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

In [20]:
# YOUR CODE HERE
wv['boy'] - wv['girl'] == wv['man'] - wv['woman']


array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [21]:
# YOUR CODE HERE
wv['queen'] - wv['king'] == wv['actress'] - wv['actor']


array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

<u><i>Some notes about Word2Vec as an internal Neural Network</i></u>:

You might wonder where does this magic comes from (at quite a low price, you just ran a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)`, actually trains a internal neural network (that you don't see).  

In a nutshell, this internal neural network predicts a word from the surroundings words in a sentences. Hence, it splits the original sentences, then for each split it chooses some words as inputs $X$ and a word as the output $y$ which it tries to predict, using the embedding space.

And as with any neural network, Word2Vec has some hyperparameters. Let's play with some of these. 

# Word2Vec hyperparameters

❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the embedding size is the one you chose.

In [23]:
# YOUR CODE HERE
# Learn a new word2vec_2 model trained on X_train,
# but with higher dimensionality (e.g. 300) and
# verify the results of the previous cells

word2vec_2 = Word2Vec(sentences=X_train, vector_size=300)
wv_2 = word2vec_2.wv

print(wv_2['dog'])


[ 1.66596733e-02  1.16798468e-01  1.34215914e-02  5.00389189e-02
 -3.50943953e-02 -1.43386200e-01  7.99065232e-02  3.68036300e-01
  1.04097417e-02 -1.06635757e-01  1.31734237e-01 -8.36066827e-02
 -3.31561528e-02  5.18345162e-02 -1.04472205e-01 -1.73157424e-01
  1.66895688e-01 -1.47067159e-01  1.39329001e-01 -2.41691303e-02
 -8.90183151e-02 -1.26114264e-02  8.18925649e-02  7.88917467e-02
  1.55001327e-01 -2.71258614e-04 -2.49591574e-01 -5.04155159e-02
 -6.89708740e-02 -1.61733299e-01 -1.05394050e-03 -4.44410555e-02
  1.72393490e-02  3.30980085e-02 -8.71504471e-02  9.81528535e-02
  5.96156679e-02 -2.96268970e-01 -7.45447427e-02  4.16836627e-02
 -3.75565104e-02  1.07706733e-01  1.30054196e-02 -1.16639055e-01
  8.29285458e-02  2.13842347e-01 -1.82461198e-02  1.00964159e-02
 -3.62771414e-02  1.85983628e-01  6.65237904e-02  1.67970862e-02
 -8.41313079e-02  1.48735523e-01 -1.89405624e-02  1.21222004e-01
  1.75024904e-02 -4.45172898e-02 -1.39016584e-02  2.73078699e-02
 -2.02724300e-02 -3.92596

❓ **Question** ❓ Use the **`Word2Vec.wv.key_to_index`** attribute to display the size of the learned vocabulary. Compare it to the number of different words in `X_train`.

In [24]:
# YOUR CODE HERE
# Use wv_2.key_to_index to get the index of 'dog' and
# compare it to number of different words in X_train

print(wv_2.key_to_index['dog'])
print(len(set([word for sentence in X_train for word in sentence])))
print(len(wv_2.key_to_index))


1185
30419
8006


There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been trained on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec:  `min_count`. 

`min_count` is a integer that tells you how many occurrences a given word should have to be learned in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to learn a representation of words that are sufficiently present in the corpus to have a robust embedded representation.

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vecs that you have trained (you can choose any `vector_size` you want).

In [42]:
# YOUR CODE HERE
# Learn a new word2vec_3 model trained on X_train, with a min_count higher than 5 and
# a word2vec_4 with a min_count smaller than 5, and
# compare the size of the vocabulary of the two models

word2vec_3 = Word2Vec(sentences=X_train, min_count=5)
wv_3 = word2vec_3.wv

word2vec_4 = Word2Vec(sentences=X_train, min_count=1)
wv_4 = word2vec_4.wv

print(len(wv_3.key_to_index))
print(len(wv_4.key_to_index))


8006
30419


Remember that Word2Vec has an internal neural network that is optimized based on some predictions. These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the Word2Vec with different `window` sizes.

❓ **Question** ❓ Train a new `word2vec_5` model with a `window` different than previously (default is 5).

In [43]:
# YOUR CODE HERE
# Learn a new word2vec_5 model trained on X_train, with a window size of 5

word2vec_5 = Word2Vec(sentences=X_train, window=5)

wv_5 = word2vec_5.wv

print(len(wv_5.key_to_index))


8006


The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the ones that you should start playing with to get a better performance for your model.

But you can also look at other arguments in the [**📚 Documentation - gensim.models.word2vec.Text8Corpus**](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)

# Convert our train and test set to RNN-ready datasets

Remember that `Word2Vec` is the first step to the overall process of feeding such a representation into a RNN, as shown here:

<img src="word2vec_representation.png" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be fed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (make sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- For the <font color=orange>test</font> set, this is understandable: <font color=orange>some words were not</font> in the <font color=blue>train</font> set and thus, their <font color=orange>embedded representation is unknown</font>
- for the <font color=blue>train set</font>, due to `min_count` hyperparameter, not all the words have a vector representation.

In any case, just skip the missing words here.

In [44]:
import numpy as np

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']

def embed_sentence(word2vec, sentence):
    # Given a sentence and a word2vec model, embeds the sentence
    # that is embed each word one after the other and returns the
    # resulting matrix as 2D numpy array
    # If a word is not in the vocabulary, it is skipped

    embed = []
    for word in sentence:
        if word in word2vec.wv:
            embed.append(word2vec.wv[word])
    return np.array(embed)

word2vec = Word2Vec(sentences=X_train)
### Checks
embedded_sentence = embed_sentence(word2vec, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))


❓ **Question** ❓ Write a function that, given a list of sentences (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

💡 _Hint_: Use the previous function `embed_sentence`

In [45]:
def embedding(word2vec, sentences):
    # Given a list of sentences, return a list of embedded sentences such that
    # each sentence is a matrix
    # If a word is not in the vocabulary, it is skipped

    return [embed_sentence(word2vec, sentence) for sentence in sentences]


X_train = embedding(word2vec, X_train)
X_test = embedding(word2vec, X_test)


❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad your sequences so you have tensors which can be divided into batches (of `batch_size`) during the optimization. Store the padded values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [46]:
### YOUR CODE HERE
# Pad the sentences in X_train and X_test to have the same length
# (use the pad_sequences function from keras.preprocessing.sequence)

from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_pad = pad_sequences(X_train, padding='pre')
X_test_pad = pad_sequences(X_test, padding='pre')


assert(len(X_train_pad.shape) == 3)
assert(len(X_test_pad.shape) == 3)
assert(X_train_pad.shape[2] == 100)
assert(X_test_pad.shape[2] == 100)




🏁 Congratulations, you are now able to use `Word2Vec` to embed your words :)

💾 Don't forget to git add/commit/push your notebook...

🚀 ... and move on to the next challenge!
